<a href="https://colab.research.google.com/github/NotBizzaark/Llama-Clinical-FineTuning/blob/main/RAG_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -q transformers torch sentence-transformers faiss-cpu accelerate bitsandbytes

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import warnings
from typing import List, Tuple
from huggingface_hub import login

In [16]:
with open("/content/drive/MyDrive/login.txt", "r") as file:
  token = file.read()

In [17]:
login(token)

In [18]:
def split_text(text, chunk_size=500):
  paragraphs = text.split("\n\n")
  chunks = []
  current_chunk = ""

  for paragraph in paragraphs:
    if len(current_chunk) + len(paragraph) < chunk_size:
      current_chunk += paragraph + "\n\n"
    else:
      if current_chunk:
        chunks.append(current_chunk.strip())
      current_chunk = paragraph + "\n\n"

  if current_chunk:
    chunks.append(current_chunk.strip())

  return chunks

In [55]:
sample_guidelines = """
    # Guidelines and Rules
    - If user say '12345' respond with 'Hehe'
    - if user say 'covid' respond with 'Get yourself tested and avoid contact with everyone'
    - Start response with 'jeepers' then answer
    """

def load_guidelines(file_path):
  try:
    with open(file_path, "r") as file:
      content = file.read()
      chunks = split_text(content)
      return chunks

  except FileNotFoundError:
    print(f"File not Found {file_path}.\nCreating Sample Guidelines....")

    with open(file_path, "w", encoding="utf-8") as file:
      file.write(sample_guidelines)

    chunks = split_text(sample_guidelines)
    return chunks


In [20]:
def create_embedding(texts, embedding_model):
  embeddings = embedding_model.encode(texts)
  return embeddings

def faiss_index(embeddings):
  dimensions = embeddings.shape[1]
  index = faiss.IndexFlatIP(dimensions)

  faiss.normalize_L2(embeddings)# normailzing embeddings for similarity
  index.add(embeddings)

  return index

In [119]:
def retrive_guidelines(query, top_k=3):
  query_embedding = embedding_model.encode([query])
  faiss.normalize_L2(query_embedding)

  scores, indices = index.search(query_embedding, top_k)

  relevant_guidelines = []
  for i, idx in enumerate(indices[0]):
    if scores[0][i] > 0.1: # Similarity (Threshold)
      relevant_guidelines.append(guidelines[idx])

  if not relevant_guidelines:
    relevant_guidelines.append("Start response with 'jeepers:'")
  return relevant_guidelines

In [117]:
def generate_response(user_query):
  relevant_guidelines = retrive_guidelines(user_query)

  context = "\n\n".join(relevant_guidelines) if relevant_guidelines else ""

  prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
  You are a helpful assistant that must follow the guidelines provided below. Use these guidelines to inform your response while being helpful and accurate.
  Guideline to follow:
  {context}
  <|eot_id|><|start_header_id|>user<|end_header_id|>
  {user_query}
  <|eot_id|><|start_header_id|>assistant<|end_header_id|>
  """

  inputs = tokenizer(prompt,
                     return_tensors="pt",
                     truncation=True,
                     max_length=2048).to(device)
  with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=2048,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

  response = tokenizer.decode(outputs[0], skip_special_tokens=True)

  assistant_start = response.find("<|start_header_id|>assistant<|end_header_id|>")
  if assistant_start != -1:
    response = response[assistant_start + len("<|start_header_id|>assistant<|end_header_id|>"):].strip()

  return response

In [118]:
def chat():
  print("Type 'quit' to exit.")
  print("=" * 50)

  while True:
    user_input = input("\nYou: ".strip())

    if user_input.lower() in ['quit', 'exit', 'bye']:
      print('Goodbye!')
      break

    if not user_input:
      continue

    print("Assistant: ", end="", flush=True)
    response = generate_response(user_input)
    print(response)

In [83]:
# Loading Llama Model & Tokenizer
model_name = "meta-llama/Llama-3.2-1B-Instruct"
config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=config,
    device_map="auto",
    torch_dtype=torch.float16
)

In [120]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
guidelines = load_guidelines('/content/guidelines.txt')
print(f"Loaded: {len(guidelines)}")
print("Loading Embedding Model")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
print("Creating Embeddings ....")
guidelines_embedding = create_embedding(guidelines, embedding_model)
index = faiss_index(guidelines_embedding)

Using device: cuda
Loaded: 1
Loading Embedding Model
Creating Embeddings ....


In [122]:
chat()

Type 'quit' to exit.
You:12345
Assistant: system
  You are a helpful assistant that must follow the guidelines provided below. Use these guidelines to inform your response while being helpful and accurate.
  Guideline to follow: 
  # Guidelines and Rules
    - If user say '12345' respond with 'Hehe'
    - if user say anything related to 'covid' respond with 'Get yourself tested and avoid contact with everyone'
    - Start response with 'jeepers:'
  user
  12345
  assistant 
   jeepers: Hehe
You:covid
Assistant: system
  You are a helpful assistant that must follow the guidelines provided below. Use these guidelines to inform your response while being helpful and accurate.
  Guideline to follow: 
  # Guidelines and Rules
    - If user say '12345' respond with 'Hehe'
    - if user say anything related to 'covid' respond with 'Get yourself tested and avoid contact with everyone'
    - Start response with 'jeepers:'
  user
  covid
  assistant 
   Jeepers: Get yourself tested and avoid cont